In [71]:
import functions

#complete_text = functions.extract_all_text(['papers/1801.00862/NISQ-Quantum.tex']) 
complete_text = functions.extract_all_text(['papers/2201.08194/BP_shadows.tex']) 
#complete_text = functions.extract_all_text(['papers/2002.12902/main.tex']) 


In [84]:

# cute the text between \begin{document} and \begin{bibliography}
complete_text2 = complete_text.split('\n')#[complete_text.find(r'\begin{document}')+len(r'\begin{document}'):complete_text.find(r'\end{document}')]
#complete_text3 = complete_text[complete_text.find(r'\begin{document}')+len(r'\begin{document}'):complete_text.find(r'\end{document}')].split('\n')

possible_keywords = ["\\title", "\\author", "\\email", "\\thanks","\\affiliation","\\date","\\input"]
temp_possible_keywords = possible_keywords.copy()
#add \t to each possible_keywords
for keyword in possible_keywords:
    temp_possible_keywords.append('\t'+keyword)
possible_keywords = tuple(temp_possible_keywords)


def content_in_pharentesis(first_division,complete_text,l):
    second_division = first_division.split('}')
    if len(second_division)==1:
        return second_division[0] + content_in_pharentesis(complete_text[l+1],complete_text,l+1)
    else:
        return second_division[0]

def extract_begin_to_end(complete_text,l, keyword):
    #add lines from complete_text, starting from l+1 until you find a line starting with \end{
    #code:
    if complete_text[l+1].startswith(keyword):
        return ''
    elif complete_text[l+1].startswith('%'):
        return extract_begin_to_end(complete_text,l+1,keyword)
    else:
        return complete_text[l+1]  +extract_begin_to_end(complete_text,l+1,keyword)

def loop_over(segments, opening, closing):
    if closing in segments[0]:
        return segments[0].split(closing)[0]
    else:
        return segments[0] +opening+ segments[1] + loop_over(segments[2:],opening, closing)


text_sections = {}
text_sections['pre_section'] = []
text_keys = {}
text_keys['plain text'] = []
in_document = False
in_section = False

for l,line in enumerate(complete_text2):
        
    
    if line.startswith(possible_keywords):
            first_division = line.split('{')
            keyword = first_division[0].replace('\\','')
            content = content_in_pharentesis(first_division[1],complete_text2,l)
            if keyword not in text_keys:
                text_keys[keyword] = [content]
            else:
                text_keys[keyword].append(content)

    elif line.startswith("\\begin{"):
        in_document = True
        first_division = line.split('{')
        second_division = first_division[1].split('}')
        keyword = second_division[0]
        content = extract_begin_to_end(complete_text2,l,'\end{'+keyword)
        if keyword not in text_keys:
            text_keys[keyword] = [content]
        else:
            text_keys[keyword].append(content)
    #print(r"{}".format(line))
    if line.startswith(("\\section","\\subsection")):
        in_section = True
        sections_started = True
        # get the name and content of the section
        first_division = line.split('{')
        keyword = loop_over(segments=first_division[1:], opening='{', closing= '}' )
        #get all the lines until the next \section or \subsection
        content = extract_begin_to_end(complete_text2,l,("\\section","\\subsection","\\begin{thebibliography}","\\end{document}"))
        keyword = r"{}".format(keyword)
        if keyword not in text_sections:
            text_sections[keyword] = [content]
        else:
            text_sections[keyword].append(content)
    if line.startswith("\\end{document}"):
        in_document = False
        in_section = False

    if in_document and not line.startswith(("\\","\t","%"," "*2," "*3," "*4," "*5)):
        if line != '':
            text_keys['plain text'].append(line)
            if not in_section:
                text_sections['pre_section'].append(line)
print(text_keys.keys())



dict_keys(['plain text', 'document', 'title', 'author', 'thanks', 'email', 'affiliation', 'date', 'abstract', 'equation', 'figure*', 'algorithm', 'algorithmic', 'theorem', 'definition', 'corollary', 'proof', 'figure', 'equation*', 'enumerate', 'align', 'align*', 'cases', 'split', 'widetext', 'thebibliography'])


In [85]:
text_sections.keys()

dict_keys(['pre_section', 'Introduction', 'Avoiding barren plateaus in variational quantum optimization', 'Variational quantum eigensolver', 'Barren plateaus and entanglement', 'Weak barren plateaus and improved algorithm', 'Weak barren plateaus and initialization of VQE', 'Definition and relation to barren plateaus ', 'Illustration of WBP-free initialization', 'Entanglement control during optimization', 'Bounding entanglement increase at a single optimization step', 'Optimization performance with learning rate', 'Summary and Discussion', 'Acknowledgments', 'Appendix', 'Classical shadows and implementation details', 'Data acquisition via classical shadows', 'Estimating subsystem purities', 'Estimating gradients', 'Example of error accumulation in an Ising model', 'Proof of Theorem~\\ref{thm:linear-shadows} \\labelSec:proof', 'Unitary \\texorpdfstring{$t$}--designs}\\labelappx:t-design', 'Entanglement and unitary \\texorpdfstring{$2$}--designs}\\labelapp:ent', 'Bounding the expected tra

In [86]:
text_sections['pre_section'] 

['24 quai Ernest-Ansermet, 1211 Geneva, Switzerland}',
 'Variational quantum algorithms are promising algorithms for achieving quantum advantage on near-term devices. The quantum hardware is used to implement a variational wave function and measure observables, whereas the classical computer is used to store and update the variational parameters. ',
 'The optimization landscape of expressive variational ans\\"atze is however dominated by large regions in parameter space, known as barren plateaus, with vanishing gradients which prevents efficient optimization. In this work we propose a general algorithm to avoid barren plateaus in the initialization and throughout the optimization. To this end we define a notion of \\emph{weak barren plateaus} (WBP) based on the entropies of local reduced density matrices. The presence of WBPs can be efficiently quantified using recently introduced shadow tomography of the quantum state with a classical computer. We demonstrate that avoidance of WBPs su

In [43]:
text_sections.keys()

dict_keys(['plain text'])

In [136]:
text_sections['Entanglement growth and learning rate']

['Here we detail the derivation of Eq.~(\\ref{eq:trace_learning}). We first upper bound the trace distance via\\begin{align}\\label{eq:trace-distance}\tT(\\rho_A, \\sigma_A) \\leq T(\\ket{\\psi}, \\ket{\\phi}) &= \\sqrt{1-f(\\ket{\\psi}, \\ket{\\phi})}, \\end{align}where $f$ stands for the pure state fidelity $f(\\ket{\\psi(\\bm{\\theta})}, \\ket{\\psi(\\bm{\\theta} +\\bm{\\delta})})=|\\bra{\\psi(\\bm{\\theta})} \\ket{\\psi(\\bm{\\theta} + \\bm{\\delta})}|^2$. Taylor expanding the pure state fidelity around $\\bm{\\theta}$ we get\\begin{equation}\\label{eq:fidelity-fisher}\tf(\\ket{\\psi(\\bm{\\theta})}, \\ket{\\psi(\\bm{\\theta} +\\bm{\\delta})}) = 1-\\frac{1}{4}\\bm{\\delta}^T \\mathcal{F}(\\bm{\\theta}) \\bm{\\delta} + \\mathcal{O}(\\bm{\\delta}^4),\\end{equation}where $\\mathcal{F}(\\bm{\\theta})$ is the quantum Fisher information matrix (QFIM) given by\\begin{equation}\\label{eq:qfim_def}\\mathcal{F}_{i j}(\\bm{\\theta}) = 4 \\Re{\\bra{\\partial_i \\psi} \\ket{\\partial_j \\psi} -

In [67]:
# from tex2py import tex2py

# with open('papers/2101.08448/NISQ_Review.tex') as f: data = f.read()
# toc = tex2py(data)

In [68]:
# for section in toc:
#     print(section)